In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# read in all the words
words = open('data/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])


32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [7]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [8]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [14]:
Xb.shape
Yb

tensor([ 8, 14, 15, 22,  0, 19,  9, 14,  5,  1, 20,  3,  8, 14, 12,  0, 11,  0,
        26,  9, 25,  0,  1,  1,  7, 18,  9,  3,  5,  9,  0, 18])

In [10]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3326, grad_fn=<NegBackward0>)

In [72]:
# 1. (For dlogprobs) dloss/dlogprobs = ??
#  Loss is mean of (negative of logprobs of each 'correct' indices---given by Yb) i.e., logprobs[range(n), Yb].mean()
# So loss is in the form:
# loss = -(a + b + c) / 3
# dloss/da = -1/3
# dloss/db = -1/3
# dloss/dc = -1/3
# Therefore, dloss/d(any) = -1/n
# But to compute dloss/dlogprobs, other numbers are there too:
# for other numbers that don't participate in computing loss, i.e., gradient = 0

# 2. (For dprobs) dloss/dprobs = dloss/dlogprobs * dlogprobs/dprobs = dlogprobs * (1 / probs)  # dlogx/dx = (1/x)

# 3. (For dcounts_sum_inv) dcounts_sum_inv = dprobs * dprobs/dcounts_sum_inv = dprobs * counts # dab/da = b
# but we have to careful with the shape
# when we compute probs = counts with shape (32, 27) * counts_sum_inv (with shape (32, 1))
# what's happening is the one number in each of 32 row of counts_sum_inv is broadcasted across 27 columns of counts.
# E.g.,
# c = a * b
# a = [2x2], b = [2x1]
# c =
# [a11 * b1, a12 * b1]
# [a21 * b2, a22 * b2]
# To compute c, each of b column is contributing to whole row. So, if there's a dL/dc, then we can compute dL/db by:
# dL/dc =
# [g1, g2]
# [g3, g4] -> says how L changes for each value in c changes
# Now, dc/db gives how c changes when each value in b changes
# dc / db1 =  # first row
# [a11, a12]
# [0  , 0  ]
# dc / db2 = # second row
# [0,   0  ]
# [a21, a22]
# Therefore:
# dL/db1 = dL/dc * dc/db1 -> g1 * a11 + g2 * a12
# dL/db2 = g3 * a21 + g4 * a22

# 4. (For dcounts_sum) = dcounts_sum_inv * dcounts_sum_inv/dcounts_sum = dcounts_sum_inv * (-1/counts_sum**2)
#   d(1/x)/dx = -1 * x ^ {-1 - 1} = -1 * x^{-2}

# 5. (For dcounts) = dcounts_sum * dcounts_sum/dcounts =
# print(logprobs.shape)
print(C.shape, emb.shape)

torch.Size([27, 10]) torch.Size([32, 3, 10])


In [ ]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one


dlogprobs = torch.zeros_like(logprobs); dlogprobs[range(n), Yb] = -1.0 / n
dprobs = dlogprobs * (probs ** -1)
dcounts_sum_inv = (counts * dprobs).sum(dim=1, keepdim=True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = -1.0 * (counts_sum **-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = norm_logits.exp() * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
dlogits_temp = torch.zeros_like(logits); dlogits_temp[range(n), logits.max(1, keepdim=True).indices.view(-1)] = 1.0
dlogits += dlogits_temp * dlogit_maxes
dh = dlogits @ W2.t()
dW2 = h.t() @ dlogits
db2 = dlogits.sum(dim=0, keepdim=True)
dhpreact = (1 - h ** 2) * dh
dbngain = (bnraw * dhpreact).sum(dim=0, keepdim=True)
dbnbias = dhpreact.sum(dim=0, keepdim=True)
dbnraw = bngain * dhpreact
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(dim=0, keepdim=True)
dbnvar = -0.5 * (bnvar + 1e-5) ** (-3 * 0.5) * dbnvar_inv
dbndiff2 = 1 / (n - 1) * torch.ones_like(bndiff2) * dbnvar
dbndiff += 2 * bndiff * dbndiff2
dbnmeani = -1 * (dbndiff).sum(0, keepdim=True)
dhprebn = dbndiff.clone()
dhprebn += 1 / n * torch.ones_like(hprebn) * dbnmeani
dembcat = dhprebn @ W1.t()
dW1 = embcat.t() @ dhprebn
db1 = dhprebn.sum(dim=0, keepdim=True)
demb = dembcat.reshape(emb.shape)

temp_demb = demb.view(-1, n_embd)
temp_Xb = Xb.flatten()
dC = torch.zeros_like(C)
for i, idx in enumerate(temp_Xb):
    dC[idx] += temp_demb[i]


# temp_demb = demb.sum(0) # (27, 10) *  (32, 3, 10)


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

torch.Size([96, 10]) torch.Size([96])
logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact:

In [88]:
e_counts = torch.tensor([[1, 2, 3]], dtype=float, requires_grad=True)
e_counts_sum = e_counts.sum(dim=1, keepdim=True)
e_counts_sum_inv = e_counts_sum ** -1
e_prob = e_counts * e_counts_sum_inv
sum_e_prob = (2 * e_prob[0, 0] + e_prob[0, 1] + 0.5 * e_prob[0, 2]).sum()

# lets retain grad before doing backward
for t in [e_prob, e_counts_sum_inv, e_counts_sum]:
    t.retain_grad()

sum_e_prob.backward()
# find derivative of sum_e_prob wrt e_counts
print(f'{e_prob.grad=}')
print(f'{e_counts_sum_inv.grad=}')
print(f'{e_counts_sum.grad=}')
print(f'{e_counts.grad=}')
print("--------------------------------")

with torch.no_grad():
    # Let's compute it by hand
    d_e_prob = torch.tensor([[2.0, 1.0, 0.5]]) # changes linearly
    print(f'Computed e_prob.grad = {d_e_prob}')
    d_e_counts_sum_inv = (e_counts * d_e_prob).sum(1, keepdim=True) # e_counts = [1, 3], d_e_prob = [1, 3] -> [1, 3]; we need [1, 1] => sum along dim = 1
    print(f'Computed e_counts_sum_inv.grad = {d_e_counts_sum_inv}')
    d_e_counts = e_counts_sum_inv * d_e_prob # e_sum_inv = [1]; d_e_prob = [1, 3] -> broadcast = [1, 3]
    print(d_e_counts)
    d_e_counts_sum = -e_counts_sum**-2 * d_e_counts_sum_inv
    print(f'Computed e_counts_sum.grad = {d_e_counts_sum}')
    d_e_counts += torch.ones_like(e_counts) * d_e_counts_sum
    print(f'Computed e_count.grad = {d_e_counts}')
    pass

e_prob.grad=tensor([[2.0000, 1.0000, 0.5000]], dtype=torch.float64)
e_counts_sum_inv.grad=tensor([[5.5000]], dtype=torch.float64)
e_counts_sum.grad=tensor([[-0.1528]], dtype=torch.float64)
e_counts.grad=tensor([[ 0.1806,  0.0139, -0.0694]], dtype=torch.float64)
--------------------------------
Computed e_prob.grad = tensor([[2.0000, 1.0000, 0.5000]])
Computed e_counts_sum_inv.grad = tensor([[5.5000]], dtype=torch.float64)
tensor([[0.3333, 0.1667, 0.0833]], dtype=torch.float64)
Computed e_counts_sum.grad = tensor([[-0.1528]], dtype=torch.float64)
Computed e_count.grad = tensor([[ 0.1806,  0.0139, -0.0694]], dtype=torch.float64)


In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())
